In [ ]:
# in case of using colab or any other online platform run this cell

!git clone https://github.com/Pooria90/EEG-Motor-Imagery-Analysis.git

import os
os.chdir('./EEG-Motor-Imagery-Analysis')

# Data

## Download dataset

In [1]:
from datautils import downloader

path  = 'data'
dpath = downloader(path, subjects=list(range(1,10)))

Subjects [1 2 3 4 5 6 7 8 9] are now in data


## Extracting .mat files

In [2]:
from datautils import mat_extractor

tr_name = 'A01T.mat'
te_name = 'A01E.mat'
x_train, y_train = mat_extractor(path=dpath/tr_name)
x_test , y_test  = mat_extractor(path=dpath/te_name)

print (f'*** Shapes ***\nx_train:\t{x_train.shape}\ny_train:\t{y_train.shape}')
print (f'x_test:\t\t{x_test.shape}\ny_test:\t\t{y_test.shape}')

*** Shapes ***
x_train:	(288, 22, 1000)
y_train:	(288,)
x_test:		(288, 22, 1000)
y_test:		(288,)


## cropping for augmentation

In [3]:
from datautils import cropper

x_train, y_train = cropper(x_train, y_train, window=500, step=500)
x_test , y_test  = cropper(x_test , y_test, window=500, step=500)

print (f'*** Shapes ***\nx_train:\t{x_train.shape}\ny_train:\t{y_train.shape}')
print (f'x_test:\t\t{x_test.shape}\ny_test:\t\t{y_test.shape}')

*** Shapes ***
x_train:	(576, 22, 500)
y_train:	(576,)
x_test:		(576, 22, 500)
y_test:		(576,)


## Split for validation

In [4]:
from sklearn.model_selection import train_test_split

x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train-1, test_size=0.25, random_state=216, shuffle=True)

print (x_tr.shape, x_va.shape)

(432, 22, 500) (144, 22, 500)


# EEGNet

In [5]:
import numpy as np
import torch

# adding a second dimension, becasue we have Conv2d in our structure
x_tr = np.expand_dims(x_tr, axis=1)
x_va = np.expand_dims(x_va, axis=1)

x_tr, x_va, y_tr, y_va = map(torch.tensor, [x_tr, x_va, y_tr, y_va])
print (x_tr.shape, x_va.shape)

torch.Size([432, 1, 22, 500]) torch.Size([144, 1, 22, 500])


## Simple training loop

In [6]:
from models import EEGNet
from fitting import train

if torch.cuda.is_available():
    target_device = 'cuda'
else:
    target_device = 'cpu'
    
model = EEGNet().to(target_device)
hist = train(model, x_tr, y_tr, x_va, y_va, batch_size=144, epochs=50, learning_rate=0.001, period=10)

*** Epoch: 1 ***
Train Loss: 1.3884 --- Train Acc 22.45
Valid Loss: 1.3852 --- Valid Acc: 27.08
*** Epoch: 10 ***
Train Loss: 1.2168 --- Train Acc 49.54
Valid Loss: 1.2511 --- Valid Acc: 43.75
*** Epoch: 20 ***
Train Loss: 1.0169 --- Train Acc 56.94
Valid Loss: 1.0740 --- Valid Acc: 47.22
*** Epoch: 30 ***
Train Loss: 0.9341 --- Train Acc 61.81
Valid Loss: 1.0249 --- Valid Acc: 50.69
*** Epoch: 40 ***
Train Loss: 0.8970 --- Train Acc 63.19
Valid Loss: 0.9959 --- Valid Acc: 54.17
*** Epoch: 50 ***
Train Loss: 0.8343 --- Train Acc 66.44
Valid Loss: 0.9681 --- Valid Acc: 54.17


## Training with early-stopping

In [7]:
from models import EEGNet
from fitting import train, EarlyStopping


if torch.cuda.is_available():
    target_device = 'cuda'
else:
    target_device = 'cpu'

model  = EEGNet().to(target_device)
e_stop = EarlyStopping(state=True, patience=5, attribute='acc')
hist   = train(model, x_tr, y_tr, x_va, y_va, 
               batch_size=144, epochs=100,
               learning_rate=0.001, period=1,
               er_stop=e_stop)

*** Epoch: 1 ***
Train Loss: 1.3877 --- Train Acc 25.46
Valid Loss: 1.3857 --- Valid Acc: 27.78
*** Epoch: 2 ***
Train Loss: 1.3795 --- Train Acc 26.16
Valid Loss: 1.3852 --- Valid Acc: 28.47
*** Epoch: 3 ***
Train Loss: 1.3679 --- Train Acc 31.94
Valid Loss: 1.3846 --- Valid Acc: 30.56
*** Epoch: 4 ***
Train Loss: 1.3626 --- Train Acc 35.19
Valid Loss: 1.3839 --- Valid Acc: 31.25
*** Epoch: 5 ***
Train Loss: 1.3553 --- Train Acc 34.03
Valid Loss: 1.3831 --- Valid Acc: 31.25
*** Epoch: 6 ***
Train Loss: 1.3486 --- Train Acc 34.72
Valid Loss: 1.3821 --- Valid Acc: 31.25
*** Epoch: 7 ***
Train Loss: 1.3382 --- Train Acc 35.88
Valid Loss: 1.3805 --- Valid Acc: 31.25
*** Epoch: 8 ***
Train Loss: 1.3262 --- Train Acc 39.58
Valid Loss: 1.3781 --- Valid Acc: 31.94
*** Epoch: 9 ***
Train Loss: 1.3220 --- Train Acc 38.89
Valid Loss: 1.3741 --- Valid Acc: 32.64
*** Epoch: 10 ***
Train Loss: 1.3070 --- Train Acc 42.13
Valid Loss: 1.3679 --- Valid Acc: 33.33
*** Epoch: 11 ***
Train Loss: 1.2951 --

# Deep ConvNet

In [1]:
from models import DeepNet
from torchsummary import summary

model = DeepNet()
summary(model, (1,22,500), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
 Conv2dConstrained-1          [-1, 25, 22, 496]             150
 Conv2dConstrained-2           [-1, 25, 1, 496]             575
       BatchNorm2d-3           [-1, 25, 1, 496]              50
               ELU-4           [-1, 25, 1, 496]               0
         MaxPool2d-5           [-1, 25, 1, 248]               0
           Dropout-6           [-1, 25, 1, 248]               0
 Conv2dConstrained-7           [-1, 50, 1, 244]           6,300
       BatchNorm2d-8           [-1, 50, 1, 244]             100
               ELU-9           [-1, 50, 1, 244]               0
        MaxPool2d-10           [-1, 50, 1, 122]               0
          Dropout-11           [-1, 50, 1, 122]               0
Conv2dConstrained-12          [-1, 100, 1, 118]          25,100
      BatchNorm2d-13          [-1, 100, 1, 118]             200
              ELU-14          [-1, 100,